In [22]:
import pandas as pd
import torch
import numpy as np
import torch.utils.data as Data
import os
import glob
import unicodedata
import string


In [15]:
atis=open("/data/jeff/work/learning/learning_resource/ATS/atis.input",'r')
atis_all_list=[]
with atis as atis_all:
    for line in atis_all:
        atis_all_list.append(one_hot_encode(line.strip()))
dtis=open("/data/jeff/work/learning/learning_resource/ATS/dtis.input",'r')
dtis_all_list=[]
with dtis as dtis_all:
    for line in dtis_all:
        dtis_all_list.append(one_hot_encode(line.strip()))
utis=open("/data/jeff/work/learning/learning_resource/ATS/utis.input",'r')
utis_all_list=[]
with utis as utis_all:
    for line in utis_all:
        utis_all_list.append(one_hot_encode(line.strip()))

/home/lijiefu/anaconda3/envs/pytorch_learn/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [3]:
def one_hot_encode(seq):
#    map = np.asarray([[0, 0, 0, 0],
#                      [1, 0, 0, 0],
#                      [0, 1, 0, 0],
#                      [0, 0, 1, 0],
#                      [0, 0, 0, 1]])
#	numpy version
	map=torch.tensor([[0, 0, 0, 0],
	                  [1, 0, 0, 0],
	                  [0, 1, 0, 0],
	                  [0, 0, 1, 0],
	                  [0, 0, 0, 1]])
	seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
	seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')
	#return map[np.fromstring(seq, np.int8) % 5] numpy version

	map = map[np.fromstring(seq, np.int8)%5]
	return map


In [13]:
atis_all_list=[]

with atis as atis_all:
    for line in atis_all:
        atis_all_list.append(one_hot_encode(line.strip()))
        

/home/lijiefu/anaconda3/envs/pytorch_learn/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [23]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [14]:
def findFiles(path): return glob.glob(path)
print(findFiles('*.input'))
all_tis=[]
tis_lines={}

['atis.input', 'dtis.input', 'utis.input']


In [30]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line.upper()) for line in lines]
for filename in findFiles('*.input'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_tis.append(category)
    lines = readLines(filename)
    tis_lines[category] = lines

n_categories = len(all_tis)

In [25]:
tis_lines.keys()

dict_keys(['atis', 'dtis', 'utis'])

In [26]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


In [27]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [33]:
input = letterToTensor('A')
hidden =torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [35]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_tis[category_i], category_i

print(categoryFromOutput(output))

('utis', 5)


In [37]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_tis)
    line = randomChoice(tis_lines[category])
    category_tensor = torch.tensor([all_tis.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = atis / line = AAAAAAAAACAAACTGATACTTGGGCATAGTTTAAAAAAAACTTAAAAGTGTGTGTGTGTGTGTGCGTGCGCGTGTGTGGTGGGGGGCGGGGCATTTCAATATAAATGCCTTAAACACAACCTTCCTGGCTATTTTATAAAATGGGTTGAATCTCTAAGGACACATATTTCCTAAGACCTCGACTGATACACAGAGTGCAAACTTGGGAGCCAGACTAGACTGCCTCAGAATCATGCCAGCTGTGTGGCACTGGAAAGTCACCTAATTTTGCAGCGCCTACCTTCACCTCTGTAAAATGGGGAACAATAATAACTAATTTATGGATGCTGTAAAGATCTGAAATTTTAAACTCAGTAATGCGCCTGGCACACATTAGACACGGGTGTTTCGTTTGTCTTGGCAAAGGCTTCGGGTTTTGCGACAATAGCCTTGCTGCCTCCGGAGCAAGCATCTCTCCTTGCCCCCTCTGGGGCATTTAAAAATGACCTCAATTCTACTCTCAATTATGTCCAGTTCAACAGTTACGTGCAGGTTACAAAGCCGAGGAAGCTGAGGTATCTTCTAAACTAACATAGCACGTACAGAATTTTTTTAAAGTTTGGTGGAGGAATCGGCAGCACAAGGTAAGCTGCGCTCTGTTCACAGACCTGGGAATCAGAACTCCGGCCGCCGCGGCCTCCAGGCGCCTCTCAGCTAACAGACTTCCTGCAGCAGGGGGAGGGGCGGCCAGCTCGCTGCAAAAAGCCGCAGGACCCCCTCACCCCGCCTTGCCGCCCCCGCTCCCACTCTCTGGGATGGGGAGTGCAGGGTCTAGGCGATGGCACTTACCAGCGCGAAGTGCACCAGCGCGTCGTAGCAGAGCCAGATGAGCGCCCCGCGGTCCGCCGCCCCCTGCCCGCGGCCCAGGCGCAGGCCCAGGGCGCAGCCCGCCGCCAGCAGCGCGGCGCACAGCAGCAGCGAACCGCCAGCCTCGG

category = utis / line = AATCAACCCTGAGTTTTGGTATTACTACTTTCTAGTCATGGATCTGCCTCCTCATAGGTAAAATTTGCAAAATATTACCCGCCGTGTGTGCCTCAGAACTGTTATGGATTAAGGAAGAGAACAATTTTTATGAAAACGTTTTGTAAATTATACAAGTGTGGGACATAATGTTCCCGGTAACTGTATGCAAACAATAAGACAACAAACGCAAAAGGCTTTTTAACTTGTTGCAGTTTTCCTGCCCCGTGTCCCTCCGTGAATGAGCAGCGTGCAGCGAGCGTGTGCGCACAGCTGCTGTTCTCCATTAACGCCACACTGCTCGGCTTGCGGACGGGGAAACCCGCCATCGCGCTCCTGCCGCCTGCCGAGCGAGGATCACCGCTCCACTGGGGTCAGCGGGCTGTTGACTCCCCGCCTGGGTGAAACCGGAGCTTCCGAGTCACGTGGCGCGCGAAGACAAACCTGGCTGCTCAGGGTTTCCCGGAGCTTTCTCCCAGCTTCCCGGCGATTCATATTCTGGCGCTTACCCCTCCCTTTAGGCTGGCGGAACCCAATCCCCGCCTGACAAATAAGTCCCGCCGGCCGGACGCAGTGGCTCGCGCCTGTGGTCCTAGCTGCTCGCGGTGCTGAGGCGGGAGAATCACCTGAGCCCGGGAGGTCGGGGGCTGCATGGAACCGTGATCGCACCACCGCACTCCAGCCTGGGCAACAGAGCGAGACCCTGTCTCCAAAAATAAGAATAAAGAAAAAAAAAAGAAAGAACTGCGGCCCCATTGGCCTGACTCTCCTGAACAGCCAAACTGTCTCCGACCAGTTACGGAAGAGGCGGCCTTGAGAGGGTAGTGTGCCATTGGCTCGATGTCCTGCCCCTCCGAAACCTAAGTCTTCAGCTTCCAATCAGGACTCAGCTTTGGGAAGAGCGCCACGCGGTGGGCGAGGGGATGCCCTAGTAAATTCGCAGGTCTCTTGACTCTT

In [38]:
criterion = nn.NLLLoss()

In [39]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0